<a href="https://colab.research.google.com/github/chonholee/tutorial/blob/main/bigdata/BigdataII_06_pca_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# いつものDriveマウント

from google.colab import drive
drive.mount('/content/drive/')
%cd '/content/drive/MyDrive/Lecture_BigData'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive/Lecture_BigData


## 主成分分析（PCA） 演習

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

## Matplotlib文字化け対策

In [ ]:
x = range(6)
y = np.linspace(0,5,6)

plt.plot(x, y)
plt.title("テスト")
plt.show()

In [ ]:
# 「IPA」フォントをインストール
!apt-get -y install fonts-ipafont-gothic
#!rm /root/.cache/matplotlib/fontlist-v300.json # キャッシュを削除

!pip install japanize-matplotlib
import japanize_matplotlib #matplotlib日本語化
import seaborn as sns
sns.set(font="IPAexGothic") #日本語フォント設定

In [ ]:
# タイトル確認
plt.plot(x, y)
plt.title("テスト")
plt.show()

## 応用例

プロ野球選手50名分のデータ

In [ ]:
import pandas as pd
url ='baseball.csv'
raw_data = pd.read_csv(
    url,
    thousands  = ',',
    encoding='utf-8'
)

raw_data.head()

数値データの標準化
StandardScalerを用いてデータを標準化

In [ ]:
from sklearn.preprocessing import StandardScaler
raw_data_drop = raw_data.drop(columns=['選手名','順位', 'チーム'] ) # 文字が入るとPCAできないのでcolumnの削除

# StandardScalerを用いたdatasetの標準化
scaler = StandardScaler()
scaler.fit(raw_data_drop)
scaler.transform(raw_data_drop)
raw_data_drop_normalize = pd.DataFrame(scaler.transform(raw_data_drop), columns=raw_data_drop.columns) # 標準化したdatasetをraw_data_drop_normalizeとする

raw_data_drop_normalize.head()


PCA 実行（22次元 ⇒ 3次元）

In [ ]:
from sklearn.decomposition import PCA
# raw_data_drop_normalizeを用いたPCA

#3軸でPCA classを作成する
here
#fitting、実際のPCA処理はここ
here

# PCAの結果はarrayで返ってくるのでをData.Frameにする
pca_result = pca.transform(raw_data_drop_normalize)
pca_result = pd.DataFrame(pca_result)
pca_result.columns = ['PC1', 'PC2', 'PC3'] 

pca_result.head()

クラスタリング

In [ ]:
from sklearn.cluster import KMeans
#クラスタリング
num_cluster = 4
color = ["red", "blue", "green", "orange"] # クラスタリング用の色

# KMeans
here

pca_result_cluster.labels_

In [ ]:
player_name = raw_data['選手名'] # annotation用に格納
pca_result["name"] = player_name
pca_result["cluster"] = pca_result_cluster.labels_
pca_result.head()

3D 描画

In [12]:
import matplotlib.pyplot as plt
from matplotlib.text import Annotation
from mpl_toolkits.mplot3d.proj3d import proj_transform
from mpl_toolkits.mplot3d.axes3d import Axes3D
from PIL import Image # アニメーション用
from matplotlib import animation # アニメーション用
from io import BytesIO # アニメーション用

In [13]:
class Annotation3D(Annotation):
    '''Annotate the point xyz with text s'''

    def __init__(self, s, xyz, *args, **kwargs):
        Annotation.__init__(self,s, xy=(0,0), *args, **kwargs)
        self._verts3d = xyz

    def draw(self, renderer):
        xs3d, ys3d, zs3d = self._verts3d
        xs, ys, zs = proj_transform(xs3d, ys3d, zs3d, renderer.M)
        self.xy=(xs,ys)
        Annotation.draw(self, renderer)

# annotate3Dを定義
def annotate3D(ax, s, *args, **kwargs):
    '''add anotation text s to to Axes3d ax'''

    tag = Annotation3D(s, *args, **kwargs)
    ax.add_artist(tag)

In [ ]:
# plot_3Dを定義
def plot_3D(data, angle = 50):
    # 3d plot用Figure
    fig = plt.figure(num=None, figsize=(12, 12), dpi=72)
    ax = fig.gca(projection = '3d')
    for i in range(len(data.index)):
        ax.scatter3D(data.iloc[i,0], data.iloc[i,1],data.iloc[i,2], c=color[int(data.iloc[i,4])]) # プロットの座標を指定
        annotate3D(ax, s=str(data.iloc[i,3]), xyz=(data.iloc[i,0], data.iloc[i,1],data.iloc[i,2]),
        fontsize=12,
        xytext=(-3,3),
        textcoords='offset points', ha='right',va='bottom', fontname="IPAexGothic") # annotation
    ax.view_init(30, angle)
    ax.set_xlim(data.describe().at['min', 'PC1'], data.describe().at['max', 'PC1'])
    ax.set_ylim(data.describe().at['min', 'PC2'], data.describe().at['max', 'PC2'])
    ax.set_zlim(data.describe().at['min', 'PC3'], data.describe().at['max', 'PC3'])
    ax.set_xlabel('PC1 ' + str(round(pca.explained_variance_ratio_[0]*100, 1)) + "%") # 軸に寄与率を表示
    ax.set_ylabel('PC2 ' + str(round(pca.explained_variance_ratio_[1]*100, 1)) + "%")
    ax.set_zlabel('PC3 ' + str(round(pca.explained_variance_ratio_[2]*100, 1)) + "%")
    buf = BytesIO()
    fig.savefig(buf, bbox_inches='tight', pad_inches=0.0)
    return Image.open(buf)

plot_3D(pca_result)
plt.show()

In [ ]:
# gif animationを作る場合
#images = [plot_3D(pca_result,angle) for angle in range(180)]
#images[0].save('dataset_temp/output.gif', save_all=True, append_images=images[1:], duration=100, loop=0)